In [ ]:
# load dependencies
import os
import re
import json
import tempfile
from rdkit import Chem
from rdkit.Chem import Draw
import multiprocessing
import shutil 
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcExactMolWt, CalcNumAtomStereoCenters, CalcNumRotatableBonds
import torch

# cpu availability for OMEGA/ROCS.
num_cores = multiprocessing.cpu_count()
print(f'num_cores = {num_cores}')

# sorting directories.
SH_dir = "/.../RuSH"
RE_dir = f"{SH_dir}/RE"

# ligand selection.
ligand = 'pim447'
ligand_sdf = f'{SH_dir}/data/references/{ligand}.sdf'

# prior selection.
prior = f"prior" # TLx or prior

##### !!! 
overwrite = True
##### !!! 

# parameterization.
job_name = f'RE_{ligand}_SF_{prior}'
epochs = 2000
allowance = 0.9
jacc_weight = 1.5
color_weight = 1.2

description = f'{epochs} epochs. {allowance} allowance. {jacc_weight} jaccard weight. {color_weight} color weight.'
print(description)

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/repos/Reinvent")
reinvent_env = os.path.expanduser("~/anaconda3/envs/reinvent.v3.2")
output_dir = os.path.expanduser(f"{RE_dir}/results/RL/{job_name}")


prior_path = f"{RE_dir}/models/guacamol.{prior}"
agent_path = f"{RE_dir}/models/guacamol.{prior}"

print(agent_path)

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
if overwrite:
    for root, dirs, files in os.walk(f"{RE_dir}/results/"):
        for d in dirs:
            if os.path.join(root, d) == output_dir: 
                # dont delete the old config, so we can rerun quickly if no changes are made.
                for root, dirs, files in os.walk(output_dir):
                    for d in dirs:
                        shutil.rmtree(os.path.join(root, d))
                break
                
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

def draw_smiles(smiles, names=None):
    def to_mol(smi):
        return Chem.MolFromSmiles(smi)
    if names:   
        return Draw.MolsToGridImage([to_mol(s) for s in smiles], molsPerRow=5,subImgSize=(200,200),
                                    legends=names)
    else:
        return Draw.MolsToGridImage([to_mol(s) for s in smiles], molsPerRow=5,subImgSize=(200,200))
    

# pim447
seeds = ["O=C(c1c([H])c([H])c(F)[c@@]([c@@]2c(F)c([H])c([H])c([H])c2F)n1)N(c3c([H])nc([H])c([H])c3[C@@]4([H])C([H])([H])[C@](N([H])[H])([H])C([H])([H])[C@@](C4([H])[H])([H])C([H])([H])[H])[H]"] 
seeds_decorations = [("C(*)1=C(C2C[C@H](C)C[C@H](N)C2)C=CN=C1", "Fc1c([H])c([H])c([H])c(F)c(*)1"),]
seeds_scaffolds = [ "O=C(N[*])c1nc([*])c(F)cc1" ]


i = 0
mol = Chem.MolFromSmiles(seeds[i])
print(CalcNumAtomStereoCenters(mol),CalcExactMolWt(mol),CalcNumRotatableBonds(mol)) 

draw_smiles([seeds[i], seeds_scaffolds[i], seeds_decorations[i][0], seeds_decorations[i][1]])


In [2]:
# initialize the dictionary
configuration = {
    "version": 3,                          # we are going to use REINVENT's newest release
    "run_type": "reinforcement_learning",  # other run types: "sampling", "validation",
                                           #                  "transfer_learning",
                                           #                  "scoring" and "create_model",
    "model_type": "default"
}

# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "http://127.0.0.1",          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                  # either to local logging or use a remote REST-interface
    "logging_frequency": 50,                # log every x-th steps
    "logging_path":  os.path.join(output_dir, "progress.log"), # load this folder in tensorboard
    "result_folder": os.path.join(output_dir, "results"),         # will hold the compounds (SMILES) and summaries
    "job_name": f"{job_name} {description}",                # set an arbitrary job name for identification
    "job_id": job_name                       # only relevant if "recipient" is set to "remote"
}

# add the "parameters" block
configuration["parameters"] = {}

# add a "diversity_filter"
configuration["parameters"]["diversity_filter"] =  {
    "name": "IdenticalMurckoScaffold",     # other options are: "IdenticalTopologicalScaffold", 
                                           #                    "NoFilter" and "ScaffoldSimilarity"
                                           # -> use "NoFilter" to disable this feature
    "nbmax": 30,                           # the bin size; penalization will start once this is exceeded
    "minscore": 0.4,                       # the minimum total score to be considered for binning
    "minsimilarity": 0.2                   # the minimum similarity to be placed into the same bin
}

# prepare the inception (we do not use it in this example, so "smiles" is an empty list)
configuration["parameters"]["inception"] = {
    "smiles": seeds,                       # fill in a list of SMILES here that can be used (or leave empty)
    "memory_size": 100,                    # sets how many molecules are to be remembered
    "sample_size": 10                      # how many are to be sampled each epoch from the memory
}

# set all "reinforcement learning"-specific run parameters
configuration["parameters"]["reinforcement_learning"] = {
    "prior": prior_path, # path to the pre-trained model
    "agent": agent_path, # path to the pre-trained model
    "n_steps": epochs,                     # the number of epochs (steps) to be performed; often 1000
    "sigma": 128,                          # used to calculate the "augmented likelihood", see publication
    "learning_rate": 0.0001,               # sets how strongly the agent is influenced by each epoch
    "batch_size": 64,                      # specifies how many molecules are generated per epoch
    "margin_threshold": 50                 # specify the (positive) margin between agent and prior
}

# just one component here, can add additional components for MPO.
components = {
    "RuSH" :
    {
        "component_type": "RuSH", 
        "name": "decoration conditioned distance and ROCS shape and color",   # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "specific_parameters": {
            'output_dir'  : output_dir,
            
            # the order of these molecules should be consistent.
            'database_from_smiles': False,
            'database_path'  : ligand_sdf, 
            "reference_smiles": list(zip(seeds, seeds_decorations, seeds_scaffolds)), 

            "partial_reward": 0.3,
            "allowance": allowance, # less strict fragments (azine was difficult to learn at 0.9)
            
            'oeomega_CA'  :'classic',           # see https://docs.eyesopen.com/applications/omega/omega/omega_opt_params.html
            'oeomega_rms' : 0.5,
            'n_conformers': 32,                # note exponential increase of compute time if not using a .db from file.
        
            'max_centers' : 6,            
            'max_molwt'   : 500,     
            'max_rotors'  : 10,    
            
            'ROCS_version': '2024',            # minor changes to fix bug
            'score_cutoff': 0.8,               # see https://docs.eyesopen.com/applications/rocs/rocs/rocs_opt_params.html
            'roc_maxconfs': 100,
            'roc_besthits': 500,
            'roc_timeout' : 1000, 
            
            'mcquery'     : True, # set these to False if you want to fetch individual conformer hits. 
            'nostructs'   : True, # not useful during RL, but can be handy if you want to visualize hits, analyse etc.

            'shape_weight': 1,     # does not reach 1 for exact smiles matches, and is always atleast somewhat positive (>0.2). 
            'color_weight': color_weight,   # consider adding more weight to color, as it scores generally much lower.
            'jacc_weight' : jacc_weight,
            'rocs_weight' : 1,
            'score_operator' : 'mean', 
            'num_cores': 10,          # be nice to the cpu and leave a few cores :)
      }
    }

}

# prepare the scoring function definition and add at the end
scoring_function = {
    "name": "custom_product",             
    "parallel": False,                    
    "parameters": [    
        components['RuSH'],
    ]
}

configuration["parameters"]["scoring_function"] = scoring_function

# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "RL_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

In [ ]:

# execute REINVENT from the command-line:

# %%capture captured_err_stream --no-stderr
print(f"{reinvent_env}/bin/python {reinvent_dir}/input.py {configuration_JSON_path} > {job_name}_OUT.txt 2>&1 ")

In [ ]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)